In [10]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd
pd.options.display.max_rows = 30


In [11]:
def salary_split(vacancy_income):
    vacancy_income = re.sub('(?<=\d)\s(?=\d)', '', vacancy_income)
    vacancy_income = vacancy_income.lower()
    words = re.findall('\D+', vacancy_income)
    number = re.findall('\d+',vacancy_income)
    #salary_min, salary_max, salary_currency = None, None, None
    if words:
        if words.count('-') != 0:
            salary_min = number[0]
            salary_max = number[1]
            salary_currency = words[1]
            return salary_min, salary_max, salary_currency
        elif words.count('от ') != 0:
            salary_min = number[0]
            salary_max = None
            salary_currency = words[1]
            return salary_min, salary_max, salary_currency
        elif words.count('до ') != 0:
            salary_min = None
            salary_max = number[0]
            salary_currency = words[1]
            return salary_min, salary_max, salary_currency
        elif words.count('по ') != 0:
            salary_min = None
            salary_max = None
            salary_currency = None
            return salary_min, salary_max, salary_currency
    else:
        salary_min = None
        salary_max = None
        salary_currency = None
        return salary_min, salary_max, salary_currency





In [12]:
#s = input('Введите название вакансии:' )
#l = s.split()
#print(l)
#text = '+'.join(l)
#print(text)

text='Data+scientist'
pages_fact = 0
pages = 0
vacancy_get = []
while pages == pages_fact:
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}
    main_link = 'https://hh.ru'
    response = requests.get(
        main_link + f'/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text={text}&page={pages}',
        headers=header).text
    soup = bs(response, 'lxml')
    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})
    vacancy_list = vacancy_block.find_all('div', {'data-qa': 'vacancy-serp__vacancy'})
    pages_butt = soup.find('div', {
        'class': 'bloko-column bloko-column_l-13 bloko-column_m-9 bloko-column_s-8 bloko-column_xs-4'})
    flag = pages_butt.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

    for vacancy in vacancy_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'}).getText()
        vacancy_link = main_link + vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        vacancy_income = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        vacancy_city = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText()
        vacancy_company = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'})
        if vacancy_company:
            vacancy_company = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).getText()
        else:
            vacancy_company = None

        salary_tuple = salary_split(vacancy_income)
        vacancy_data['Название вакансии'] = vacancy_name
        vacancy_data['Сайт'] = 'hh.ru'
        vacancy_data['Компания'] = vacancy_company
        vacancy_data['Город'] = vacancy_city
        vacancy_data['Минимально'] = salary_tuple[0]
        vacancy_data['Максимально'] = salary_tuple[1]
        vacancy_data['Валюта'] = salary_tuple[2]
        vacancy_data['Ссылка на вакансию'] = vacancy_link

        vacancy_get.append(vacancy_data)
    if flag:
        pages_fact += 1
    pages += 1

In [13]:
df = pd.DataFrame(vacancy_get)

In [14]:
df

,Название вакансии,Сайт,Компания,Город,Минимально,Максимально,Валюта,Ссылка на вакансию
0,Data Scientist,hh.ru,ООО КРИПТОН,Москва,120000,180000,руб.,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
1,Python Data Scientist,hh.ru,Web Interactive World,"Москва, Площадь Революции и еще 3",150000,None,руб.,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
2,Data Scientist,hh.ru,ООО Варити,"Москва, Марксистская и еще 2",130000,None,руб.,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
3,Data Scientist,hh.ru,Корпорация WCH Service Bureau,Казань,None,None,None,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
4,Data Scientist,hh.ru,ivi,"Москва, Дмитровская",None,None,None,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
5,Data Scientist,hh.ru,ЦРТ | Группа компаний,"Санкт-Петербург, Выборгская",None,None,None,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
6,Data Scientist,hh.ru,ООО Компания ТрэйдСофт,Киров (Кировская область),None,None,None,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
7,Бизнес-аналитик/Data Scientist,hh.ru,ООО Рукоделие онлайн,Нижний Новгород,40000,None,руб.,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
8,Аналитик-математик (Data scientist),hh.ru,ООО Финансово-правовой альянс,"Санкт-Петербург, Сенная площадь",80000,100000,руб.,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
9,Data Scientist,hh.ru,Prequel,Калининград,None,None,None,https://hh.ruhttps://chelyabinsk.hh.ru/vacancy...
